# Credit Value at Risk

In [1]:
import numpy as np
import pandas as pd
import requests
import io
from scipy.stats import binom, norm
from scipy.linalg import eig, inv

from math import exp
from math import log
from math import sqrt
import locale

In [2]:
_ = locale.setlocale(locale.LC_ALL, '')

### Exercise 21.4
Answer from the textbook:
* The probability of an Aaa rating staying Aaa over six months is 95.35%
* The probability of it moving to Aa is 4.40%

Retrieving Table 21.1 from the author's website:

In [3]:
idx_labels = ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca-C', 'Default']
# For the 6th edition: http://www-2.rotman.utoronto.ca/~hull/calcs/TransitionMatrixCalculations6e_Section_19_1.xls
with requests.get('http://www-2.rotman.utoronto.ca/~hull/calcs/TransitionMatrixCalculations5e.xls') as s:
    trans_mat = pd.read_excel(io.BytesIO(s.content), sheet_name='Transition Matrices',
                              header=None, index_col=None, nrows=9, usecols='F:N', names=idx_labels)
trans_mat = trans_mat.set_axis(idx_labels)

In [4]:
trans_mat

,Aaa,Aa,A,Baa,Ba,B,Caa,Ca-C,Default
Aaa,0.9094,0.0836,0.0059,0.0008,0.0002,0.0000,0.0000,0.0000,0.0000
Aa,0.0087,0.8968,0.0884,0.0045,0.0007,0.0004,0.0002,0.0000,0.0002
A,0.0006,0.0264,0.9090,0.0567,0.0051,0.0012,0.0004,0.0001,0.0006
Baa,0.0004,0.0016,0.0444,0.9016,0.0409,0.0075,0.0017,0.0002,0.0018
Ba,0.0001,0.0005,0.0047,0.0666,0.8303,0.0790,0.0078,0.0012,0.0099
B,0.0001,0.0003,0.0016,0.0051,0.0532,0.8218,0.0739,0.0061,0.0379
Caa,0.0000,0.0001,0.0003,0.0011,0.0046,0.0782,0.7852,0.0330,0.0975
Ca-C,0.0000,0.0000,0.0007,0.0000,0.0080,0.0319,0.1141,0.5128,0.3324
Default,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


To calculate the square root of the transition matrix, we need to first find the eigenvectors (denoted as X) and eighenvalues (denoted as &lambda;) of the given 1-year transition matrix. Placing the eigenvalues on a diagonal of a zero-filled matrix, will create a diagonal matrix &Lambda;.

Then the 6-month transition matrix can be calculated as follows:
$$\text{6-months-transition-matrix} = X\cdot \sqrt{\Lambda}\cdot X^{-1}$$
where $\sqrt{\Lambda}$ denotes taking an element-wise square root.

In [5]:
eigenvalues, eigenvectors = eig(trans_mat)

In [6]:
eigenvalues = eigenvalues.real
eigenvectors = eigenvectors.real
eigenvectors_inv = inv(eigenvectors)
eigenvalues_diag = np.diag(eigenvalues)

In [7]:
trans_mat_6m = pd.DataFrame(np.matmul(np.matmul(eigenvectors, np.sqrt(eigenvalues_diag)), eigenvectors_inv),
                            index=trans_mat.index, columns=trans_mat.columns)

In [8]:
trans_mat_6m

,Aaa,Aa,A,Baa,Ba,B,Caa,Ca-C,Default
Aaa,9.535189e-01,0.043984,0.002017,0.000349,0.000094,-0.000008,-0.000003,1.664241e-08,-0.000003
Aa,4.571788e-03,0.946547,0.046519,0.001637,0.000295,0.000191,0.000099,-3.312756e-06,0.000091
A,2.781931e-04,0.013883,0.952709,0.029762,0.002377,0.000528,0.000186,5.411935e-05,0.000274
Baa,2.046226e-04,0.000666,0.023309,0.948743,0.021917,0.003507,0.000803,9.470901e-05,0.000806
Ba,4.763268e-05,0.000236,0.002051,0.035745,0.910074,0.043374,0.003313,6.080880e-04,0.004603
B,5.237193e-05,0.000149,0.000795,0.002164,0.029237,0.904809,0.041123,3.233077e-03,0.018437
Caa,-1.478589e-06,0.000050,0.000130,0.000513,0.001797,0.043462,0.884279,2.054518e-02,0.049225
Ca-C,-6.489379e-07,-0.000008,0.000403,-0.000149,0.004528,0.017670,0.070879,7.150412e-01,0.191578
Default,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000


In [9]:
print(f"The probability of a company rated Aaa staying Aaa pver 6 months: {trans_mat_6m.loc['Aaa', 'Aaa']:.2%}")
print(f"The probability of it moving to Aa: {trans_mat_6m.loc['Aaa', 'Aa']:.2%}")

The probability of a company rated Aaa staying Aaa pver 6 months: 95.35%
The probability of it moving to Aa: 4.40%


### Exercise 21.6
Answer from the textbook:
* The pobability of 6 or more defaults is 0.0005

In [10]:
num_loans = 100
pr_default = .01
num_defaults = 6
print(f'Probability of {num_defaults:d} or more defaults: {(1 - binom.cdf(num_defaults-1, num_loans, pr_default)):.4f}')

Probability of 6 or more defaults: 0.0005


### Exercise 21.7
Answer from the textbook:
* The pobability of 6 or more defaults is 0.0021

In [11]:
pds = [.005, .015]
pr_default = sum([1 - binom.cdf(num_defaults-1, num_loans, pd) for pd in pds]) / len(pds)
print(f'Probability of {num_defaults:d} or more defaults: {pr_default:.4f}')

Probability of 6 or more defaults: 0.0021


### Exercise 21.8
Answer from the textbook (confirmed by the author to be incorrect):
* The autocorrelation is quite high at 0.546

Retrieving Table 11.6 from the author's website.

In [12]:
with requests.get('http://www-2.rotman.utoronto.ca/~hull/calcs/Chapter11DefaultRates5e.xls') as s:
    df = pd.read_excel(io.BytesIO(s.content), index_col=0, header=5,
                       usecols='A,C', names=['Year', 'Defaul Rate'], parse_dates=True)

In [13]:
df = df.dropna()
df = df.asfreq('AS')

In [14]:
df

,Defaul Rate
Year,
1970-01-01,0.02631
1971-01-01,0.00286
1972-01-01,0.00453
1973-01-01,0.00456
1974-01-01,0.00275
1975-01-01,0.00361
1976-01-01,0.00176
1977-01-01,0.00354
1978-01-01,0.00354


In [15]:
print(f'The autocorrelation is: {df.iloc[:,0].autocorr():.3f}')

The autocorrelation is: 0.563


### Exercise 21.9

In [16]:
notional = 100
risk_free_rate = .05 # continuous compounding
pr_downgrade = 1. - .004 - .09 - .6574
T = 1

In [17]:
price = notional * exp(-risk_free_rate * T) * pr_downgrade
print('The price of the credit derivative (using historical probabilities) is: {:s}'
     .format(locale.currency(price, grouping=True)))

The price of the credit derivative (using historical probabilities) is: $23.65


### Exercise 21.10

In [18]:
ead = 10_000_000
pr_default = .01
recovery_rate = .4
percentile = .995
ρ = .2

In [19]:
wcdr = norm.cdf((norm.ppf(pr_default) + sqrt(ρ)*norm.ppf(percentile)) / sqrt(1-ρ))
print(f'WCDR: {wcdr:.4f}')

WCDR: 0.0946


In [20]:
var = (wcdr - pr_default) * ead * (1 - recovery_rate)
print('{:.1%} {:d}-year credit VaR: {:s}'.format(percentile, T, locale.currency(var, grouping=True)))

99.5% 1-year credit VaR: $507,527.27


### Exercise 21.11

For this exercise we need to calculate:
$$\text{3-months-transition-matrix} = X\cdot \Lambda^{\frac{1}{4}}\cdot X^{-1}$$
and then multiply it by the one year transition matrix. I'll reuse the variables initialized in Exercise 21.4 above.

In [22]:
trans_mat_3m = pd.DataFrame(np.matmul(np.matmul(eigenvectors, np.power(eigenvalues_diag, 1./4)), eigenvectors_inv),
                            index=trans_mat.index, columns=trans_mat.columns)
trans_mat_1_25y = pd.DataFrame(np.matmul(trans_mat, trans_mat_3m),
                               index=trans_mat.index, columns=trans_mat.columns)

In [23]:
trans_mat_1_25y

,Aaa,Aa,A,Baa,Ba,B,Caa,Ca-C,Default
Aaa,8.881857e-01,0.101885,0.008451,0.001078,0.000259,0.000010,0.000003,7.860535e-08,0.000003
Aa,1.060930e-02,0.873227,0.107734,0.006330,0.000946,0.000514,0.000252,3.530463e-06,0.000262
A,7.747760e-04,0.032185,0.888395,0.069193,0.006566,0.001588,0.000518,1.210009e-04,0.000784
Baa,4.947861e-04,0.002154,0.054180,0.879451,0.049406,0.009629,0.002185,2.562884e-04,0.002369
Ba,1.276856e-04,0.000644,0.006209,0.080389,0.793825,0.094294,0.010347,1.493212e-03,0.012795
B,1.222865e-04,0.000376,0.002012,0.006792,0.063468,0.784304,0.087614,7.385040e-03,0.047927
Caa,1.676339e-06,0.000125,0.000398,0.001429,0.006266,0.092762,0.740984,3.704667e-02,0.120987
Ca-C,7.470702e-07,0.000008,0.000822,0.000157,0.009447,0.037897,0.128243,4.348475e-01,0.388462
Default,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000
